
# Using a Simple Custom Unit Model

This demo will show how a user can utilize a custom unit model using the WaterTAP framework. For a guide on how to create this simple unit model, see [Creating a Simple Unit Model](create_a_simple_unit_model.ipynb). For documentation on unit models that already exist in WaterTAP, see [Unit Model Documentation](https://watertap.readthedocs.io/en/latest/technical_reference/unit_models/index.html).

## Step 1: Import the necessary functions

In [1]:
from pyomo.environ import ConcreteModel, assert_optimal_termination
from pyomo.util.check_units import assert_units_consistent
from idaes.core import FlowsheetBlock
from idaes.core.util.model_statistics import degrees_of_freedom

import idaes.core.util.scaling as iscale
from watertap.core.solvers import get_solver

# Imports the property model created in the "Creating a Simple Property Model" Jupyter Notebook
%run creating_a_simple_property_model.ipynb

# To import a custom property model, custom_prop_pack, add it to the following directory and run the line below
# from watertap.property_models.custom_prop_pack import ProcessBlockClassName

# Imports the unit model created in the "Creating a Simple Unit Model" Jupyter Notebook
%run creating_a_simple_unit_model.ipynb

# To import a custom unit model, custom_unit_model, add it to the following directory and run the line below
# from watertap.unit_models.custom_unit_model import ProcessBlockClassName

## Step 2: Create the ConcreteModel and FlowsheetBlock
Create the flowsheet by attaching the property package and building the custom unit model.

In [2]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
# Attach property package
m.fs.properties = PropParameterBlock()
# Build the unit model
m.fs.filter = Filtration(property_package=m.fs.properties)

# Display model
# Note that there are the recovery and removal fraction variables are on m.fs.filter
# any variable that starts with a _ can be ignored, they are references
# note that are three separate state blocks on the model (properties_in, properties_out, properties_waste)
print("first display")
m.fs.filter.display()

first display
Block fs.filter

  Variables:
    recovery_mass_phase_comp : Size=3, Index=fs.properties.phase_list*fs.properties.component_list, Units=dimensionless
        Key             : Lower : Value : Upper : Fixed : Stale : Domain
         ('Liq', 'H2O') : 1e-08 :   0.5 :     1 : False : False :  Reals
        ('Liq', 'NaCl') : 1e-08 :   0.5 :     1 : False : False :  Reals
         ('Liq', 'TSS') : 1e-08 :   0.5 :     1 : False : False :  Reals
    removal_fraction_mass_phase_comp : Size=3, Index=fs.properties.phase_list*fs.properties.component_list, Units=dimensionless
        Key             : Lower : Value : Upper : Fixed : Stale : Domain
         ('Liq', 'H2O') : 1e-08 :   0.5 :     1 : False : False :  Reals
        ('Liq', 'NaCl') : 1e-08 :   0.5 :     1 : False : False :  Reals
         ('Liq', 'TSS') : 1e-08 :   0.5 :     1 : False : False :  Reals
    _flow_mass_phase_comp_inlet_ref : Size=3, Index=fs._time*fs.properties.phase_list*fs.properties.component_list, Referenc

## Step 3: Set the operating conditions of the unit model
Specify the feed conditions and unit model variables such that the degrees of freedom are zero. Default scaling should also be set for the flow rate to ensure the model is well-scaled.

In [3]:
# Note there are 8 degrees of freedom
print("DOF before specifying:", degrees_of_freedom(m.fs))

# Specify the feed
m.fs.filter.properties_in[0].pressure.fix(2e5)
m.fs.filter.properties_in[0].temperature.fix(273.15 + 25)
m.fs.filter.properties_in[0].flow_mass_phase_comp["Liq", "H2O"].fix(1)
m.fs.filter.properties_in[0].flow_mass_phase_comp["Liq", "NaCl"].fix(0.035)
m.fs.filter.properties_in[0].flow_mass_phase_comp["Liq", "TSS"].fix(120e-6)

# An alternative to setting the state variables at the state block is to use the port like below
# Note that the time domain 0, is now accessed with the other indices, this is the case for ports
# m.fs.filter.inlet.pressure[0].fix(2e5)
# m.fs.filter.inlet.temperature[0].fix(273.15 + 25)
# m.fs.filter.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'].fix(1)
# m.fs.filter.inlet.flow_mass_phase_comp[0, 'Liq', 'NaCl'].fix(0.035)
# m.fs.filter.inlet.flow_mass_phase_comp[0, 'Liq', 'TSS'].fix(120e-6)

# specify the recovery or removal
m.fs.filter.removal_fraction_mass_phase_comp["Liq", "TSS"].fix(0.9)
m.fs.filter.recovery_mass_phase_comp["Liq", "H2O"].fix(0.97)
m.fs.filter.recovery_mass_phase_comp["Liq", "NaCl"].fix(0.97)

# Currently the outlet pressure of the waste is unused (i.e. not used in any constraint) so it isn't counted in the
# degrees of freedom, but if we connected the waste to another unit model then the pressure would be used.
# So in reality, the unit model has 9 DOF and the last one is fixed here.
m.fs.filter.properties_waste[0].pressure.fix(101325)
print("DOF after specifying:", degrees_of_freedom(m.fs))

# The user should provide the scale for the flow rate, so that our tools can ensure the model is well scaled
# Generally, scaling factors should be such that if it is multiplied by the variable it will range between 0.01 and 100
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling(
    "flow_mass_phase_comp", 1e2, index=("Liq", "NaCl")
)
m.fs.properties.set_default_scaling(
    "flow_mass_phase_comp", 1e4, index=("Liq", "TSS")
)
iscale.calculate_scaling_factors(m.fs)  # this utility scales the model


DOF before specifying: 8
DOF after specifying: 0


## Step 4: Solve the flowsheet and display results

In [4]:
# Check that units are consistent
assert_units_consistent(m) 

# Check that the degrees of freedom are what we expect
assert (
    degrees_of_freedom(m) == 0
)  

solver = get_solver()
results = solver.solve(m, tee=False)

# Check that the solver finds an optimal solution
assert_optimal_termination(results)

# Display results
print("second display")
m.fs.filter.display()

second display
Block fs.filter

  Variables:
    recovery_mass_phase_comp : Size=3, Index=fs.properties.phase_list*fs.properties.component_list, Units=dimensionless
        Key             : Lower : Value               : Upper : Fixed : Stale : Domain
         ('Liq', 'H2O') : 1e-08 :                0.97 :     1 :  True :  True :  Reals
        ('Liq', 'NaCl') : 1e-08 :                0.97 :     1 :  True :  True :  Reals
         ('Liq', 'TSS') : 1e-08 : 0.09999999999999998 :     1 : False : False :  Reals
    removal_fraction_mass_phase_comp : Size=3, Index=fs.properties.phase_list*fs.properties.component_list, Units=dimensionless
        Key             : Lower : Value                : Upper : Fixed : Stale : Domain
         ('Liq', 'H2O') : 1e-08 : 0.030000000000000027 :     1 : False : False :  Reals
        ('Liq', 'NaCl') : 1e-08 : 0.030000000000000027 :     1 : False : False :  Reals
         ('Liq', 'TSS') : 1e-08 :                  0.9 :     1 :  True :  True :  Reals
    _fl